In [ ]:
pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 
from selenium.webdriver.support import expected_conditions as EC

import json

In [ ]:
driver = webdriver.Chrome()
driver.get("https://www.gmarket.co.kr/")

In [ ]:
# 로그인 진행
with open('gmarket_id.json', 'r') as json_file:
    data = json.load(json_file)
    username = data.get('username')
    password = data.get('password')

login_link = driver.find_element(By.LINK_TEXT, "로그인")
login_link.click()

id_input = driver.find_element(By.ID, "typeMemberInputId")
password_input = driver.find_element(By.ID, "typeMemberInputPassword")

id_input.clear()
id_input.send_keys(username)

password_input.clear()
password_input.send_keys(password)


login_button = driver.find_element(By.ID, "btn_memberLogin")
login_button.click()
